# Load RoITD dataset

In [1]:
from langchain_ollama import ChatOllama
from datasets import load_dataset
import json


from tqdm.auto import tqdm
from functools import cache

In [2]:
ds_roitd = load_dataset("dragosnicolae555/RoITD")
ds_roitd['train'][900]

{'version': 'v2.0',
 'title': 'IT1',
 'paragraph': 'Aplicatia de fitness din ceas va permite sa urmariti in orice moment progresul dvs. catre obiectivele dvs. Aplicatia de fitness este accesibila de pe ecranul de pornire al ceasului dvs. si ofera o gazda o activitate urmarita. Alergarea, mersul cu bicicleta, mersul pe jos, inotul, inclusiv sporturile preferate, cum ar fi fotbalul, baschetul, voleiul si antrenamentul imens popular care cuprinde globul.',
 'question': 'De unde este accesibilă aplicația meteo?',
 'id': 'HL57XJ9615',
 'answer': 'de pe ecranul de pornire al ceasului',
 'answer_start': '145',
 'is_impossible': 'FALSE',
 'Unnamed: 8': None,
 'Unnamed: 9': None}

# Keep only possible questions

In [3]:
ds_roitd

DatasetDict({
    train: Dataset({
        features: ['version', 'title', 'paragraph', 'question', 'id', 'answer', 'answer_start', 'is_impossible', 'Unnamed: 8', 'Unnamed: 9'],
        num_rows: 9157
    })
})

In [4]:
ds_roitd = ds_roitd.filter(lambda x: x['is_impossible'] == 'FALSE')
ds_roitd

DatasetDict({
    train: Dataset({
        features: ['version', 'title', 'paragraph', 'question', 'id', 'answer', 'answer_start', 'is_impossible', 'Unnamed: 8', 'Unnamed: 9'],
        num_rows: 4417
    })
})

# Create new list

In [5]:
ds_roitd_ollama = []

# Augument using llama 3.2

In [6]:
@cache
def get_ollama_chat(temp = 0.2):
    return ChatOllama(
        model="llama3.2",
        temperature=temp,
        max_retries=3,
    )


In [7]:
messages = [
    (
        "system",
        """Scurtează sau lungește RĂSPUNS doar extrăgând numai cuvinte alăturate din CONTEXT. Nu copia RĂSPUNS.
        
        """,
    ),
    ("human", """
        Va trebui doar reformulat câmpul RĂSPUNS, nimic altceva. Răspunde doar cu reformularea răspunsului. Extrage strict cuvinte din CONTEXT.
        CONTEXT: '$ctxt$'
        ÎNTREBARE: '$ques$'
        RĂSPUNS: '$ans$'
        RĂSPUNS REFORMULAT: """
    ),
]

In [8]:
improved = 0

for item in tqdm(ds_roitd['train']):
    for temp in [0.1, 0.2, 0.3]:
        ctxt = item['paragraph']
        ques = item['question']
        ans = item['answer']
        ans_start = item['answer_start']
        id = item['id']
        
        llm = get_ollama_chat(temp)
        msg = messages.copy()
        # print(msg[1][1])
        msg[1] = ("human", msg[1][1].replace("$ctxt$", ctxt).replace("$ques$", ques).replace("$ans$", ans))
        # print(msg[1][1])
        
        ans_ollama = llm.invoke(msg)
        
        ans_ollama = ans_ollama.content.strip().strip('.').strip()
        
        ans = [ans]
        ans_start = [ans_start]
        
        if ans_ollama != ans[0].strip().strip('.').strip() and ctxt.find(ans_ollama) != -1:
            ans.append(ans_ollama)
            ans_start.append(ctxt.find(ans_ollama))
            
            improved += 1
            print(improved)
        else:
            continue
        
        ds_roitd_ollama.append({
            'id': id,
            'context': ctxt,
            'question': ques,
            'answers': {'text': ans, 'answer_start': ans_start},
        })
        break

  0%|          | 0/4417 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154


In [9]:
print(f"Improved {improved} answers using llama 3.2")

Improved 154 answers using llama 3.2


In [10]:
with open('ds_roitd_ollama_added.json', 'w') as f:
    json.dump(ds_roitd_ollama, f, indent=4)
with open('../ds_roitd_ollama_added.json', 'w') as f:
    json.dump(ds_roitd_ollama, f, indent=4)

In [11]:
import pandas as pd

df = pd.DataFrame(ds_roitd_ollama)
df.to_csv('ds_roitd_ollama_added.csv')

In [13]:
for i in range(len(ds_roitd_ollama)):
    for j in range(len(ds_roitd_ollama[i]['answers']['answer_start'])):
        if type(ds_roitd_ollama[i]['answers']['answer_start'][j]) == str:
            ds_roitd_ollama[i]['answers']['answer_start'][j] = int(ds_roitd_ollama[i]['answers']['answer_start'][j])

In [14]:
df = pd.DataFrame(ds_roitd_ollama)
df.to_csv('ds_roitd_ollama_added.csv')

In [17]:
ds_old = []

for item in tqdm(ds_roitd['train']):
    ctxt = item['paragraph']
    ques = item['question']
    ans = item['answer']
    ans_start = item['answer_start']
    id = item['id']
    ans = [ans]
    ans_start = [int(ans_start)]
        
    ds_old.append({
        'id': id,
        'context': ctxt,
        'question': ques,
        'answers': {'text': ans, 'answer_start': ans_start},
    })

df_old = pd.DataFrame(ds_old)
df_old_filtered = df_old[~df_old['id'].isin(df['id'])]

df_roitd_final = pd.concat([df, df_old_filtered], ignore_index=True)

df_roitd_final.to_csv('ds_roitd_ollama_final.csv')

  0%|          | 0/4417 [00:00<?, ?it/s]

In [21]:
results_gen = {
    "prompt_1":{
        0.1: 0,
        0.2: 0,
        0.3: 0,
    },
    "prompt_2":{
        0.1: 0,
        0.2: 0,
        0.3: 0,
    },
}

prompt_1 = [
    (
        "system",
        """Scurtează sau lungește RĂSPUNS doar extrăgând numai cuvinte alăturate din CONTEXT. Nu copia RĂSPUNS.
        
        """,
    ),
    ("human", """
        Va trebui doar reformulat câmpul RĂSPUNS, nimic altceva. Răspunde doar cu reformularea răspunsului. Extrage strict cuvinte din CONTEXT.
        CONTEXT: '$ctxt$'
        ÎNTREBARE: '$ques$'
        RĂSPUNS: '$ans$'
        RĂSPUNS REFORMULAT: """
    ),
]

prompt_2 = [
    (
        "system",
        """Scurtează sau lungește [RĂSPUNS] doar extrăgând numai cuvinte alăturate din [CONTEXT]. Nu copia [RĂSPUNS].
        
        """,
    ),
    ("human", """
        Va trebui doar reformulat câmpul [RĂSPUNS], nimic altceva. Răspunde doar cu reformularea răspunsului. Extrage strict cuvinte din [CONTEXT].
        [CONTEXT]: '$ctxt$'
        [ÎNTREBARE]: '$ques$'
        [RĂSPUNS]: '$ans$'
        [RĂSPUNS REFORMULAT]: """
    ),
]

for item in tqdm(ds_roitd_ollama):
    # Prompt 1
    for temp in [0.1, 0.2, 0.3]:
        ctxt = item['context']
        ques = item['question']
        ans = item['answers']['text'][0]
        
        llm = get_ollama_chat(temp)
        
        msg = prompt_1.copy()
        msg[1] = ("human", msg[1][1].replace("$ctxt$", ctxt).replace("$ques$", ques).replace("$ans$", ans))
        
        ans_ollama = llm.invoke(msg)
        
        ans_ollama = ans_ollama.content.strip().strip('.').strip()
        
        
        if ans_ollama != ans[0].strip().strip('.').strip() and ctxt.find(ans_ollama) != -1:
            results_gen['prompt_1'][temp] += 1
            print(results_gen)
            
    # Prompt 2
    for temp in [0.1, 0.2, 0.3]:
        ctxt = item['context']
        ques = item['question']
        ans = item['answers']['text'][0]
        
        llm = get_ollama_chat(temp)
       
        msg = prompt_2.copy()
        msg[1] = ("human", msg[1][1].replace("$ctxt$", ctxt).replace("$ques$", ques).replace("$ans$", ans))
        
        
        ans_ollama = llm.invoke(msg)
        
        ans_ollama = ans_ollama.content.strip().strip('.').strip()
        
        
        if ans_ollama != ans[0].strip().strip('.').strip() and ctxt.find(ans_ollama) != -1:
            results_gen['prompt_2'][temp] += 1
            print(results_gen)

  0%|          | 0/154 [00:00<?, ?it/s]

{'prompt_1': {0.1: 1, 0.2: 0, 0.3: 0}, 'prompt_2': {0.1: 0, 0.2: 0, 0.3: 0}}
{'prompt_1': {0.1: 1, 0.2: 1, 0.3: 0}, 'prompt_2': {0.1: 0, 0.2: 0, 0.3: 0}}
{'prompt_1': {0.1: 1, 0.2: 1, 0.3: 0}, 'prompt_2': {0.1: 1, 0.2: 0, 0.3: 0}}
{'prompt_1': {0.1: 1, 0.2: 1, 0.3: 0}, 'prompt_2': {0.1: 1, 0.2: 1, 0.3: 0}}
{'prompt_1': {0.1: 1, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 1, 0.2: 1, 0.3: 0}}
{'prompt_1': {0.1: 2, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 1, 0.2: 1, 0.3: 0}}
{'prompt_1': {0.1: 2, 0.2: 2, 0.3: 1}, 'prompt_2': {0.1: 1, 0.2: 1, 0.3: 0}}
{'prompt_1': {0.1: 2, 0.2: 2, 0.3: 2}, 'prompt_2': {0.1: 1, 0.2: 1, 0.3: 0}}
{'prompt_1': {0.1: 2, 0.2: 2, 0.3: 2}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 0}}
{'prompt_1': {0.1: 3, 0.2: 2, 0.3: 2}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 0}}
{'prompt_1': {0.1: 3, 0.2: 3, 0.3: 2}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 0}}
{'prompt_1': {0.1: 3, 0.2: 3, 0.3: 3}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 0}}
{'prompt_1': {0.1: 3, 0.2: 3, 0.3: 3}, 'prompt_2': {0.1: 2, 0.2: 2, 0.3: 0}}

In [22]:
print(results_gen)

{'prompt_1': {0.1: 90, 0.2: 95, 0.3: 81}, 'prompt_2': {0.1: 62, 0.2: 68, 0.3: 61}}


In [23]:
df_roitd_not_ollama = df_roitd_final.loc[~df_roitd_final['id'].isin(df['id'])]
df_roitd_not_ollama = df_roitd_not_ollama.sample(n=154)
ds_roitd_not_ollama = df_roitd_not_ollama.to_dict(orient='records')

In [25]:
results_nongen = {
    "prompt_1":{
        0.1: 0,
        0.2: 0,
        0.3: 0,
    },
    "prompt_2":{
        0.1: 0,
        0.2: 0,
        0.3: 0,
    },
}

prompt_1 = [
    (
        "system",
        """Scurtează sau lungește RĂSPUNS doar extrăgând numai cuvinte alăturate din CONTEXT. Nu copia RĂSPUNS.
        
        """,
    ),
    ("human", """
        Va trebui doar reformulat câmpul RĂSPUNS, nimic altceva. Răspunde doar cu reformularea răspunsului. Extrage strict cuvinte din CONTEXT.
        CONTEXT: '$ctxt$'
        ÎNTREBARE: '$ques$'
        RĂSPUNS: '$ans$'
        RĂSPUNS REFORMULAT: """
    ),
]

prompt_2 = [
    (
        "system",
        """Scurtează sau lungește [RĂSPUNS] doar extrăgând numai cuvinte alăturate din [CONTEXT]. Nu copia [RĂSPUNS].
        
        """,
    ),
    ("human", """
        Va trebui doar reformulat câmpul [RĂSPUNS], nimic altceva. Răspunde doar cu reformularea răspunsului. Extrage strict cuvinte din [CONTEXT].
        [CONTEXT]: '$ctxt$'
        [ÎNTREBARE]: '$ques$'
        [RĂSPUNS]: '$ans$'
        [RĂSPUNS REFORMULAT]: """
    ),
]

for item in tqdm(ds_roitd_not_ollama):
    # Prompt 1
    for temp in [0.1, 0.2, 0.3]:
        ctxt = item['context']
        ques = item['question']
        ans = item['answers']['text'][0]
        
        llm = get_ollama_chat(temp)
        
        msg = prompt_1.copy()
        msg[1] = ("human", msg[1][1].replace("$ctxt$", ctxt).replace("$ques$", ques).replace("$ans$", ans))
        
        ans_ollama = llm.invoke(msg)
        
        ans_ollama = ans_ollama.content.strip().strip('.').strip()
        
        
        if ans_ollama != ans[0].strip().strip('.').strip() and ctxt.find(ans_ollama) != -1:
            results_nongen['prompt_1'][temp] += 1
            print(results_nongen)
            
    # Prompt 2
    for temp in [0.1, 0.2, 0.3]:
        ctxt = item['context']
        ques = item['question']
        ans = item['answers']['text'][0]
        
        llm = get_ollama_chat(temp)
       
        msg = prompt_2.copy()
        msg[1] = ("human", msg[1][1].replace("$ctxt$", ctxt).replace("$ques$", ques).replace("$ans$", ans))
        
        
        ans_ollama = llm.invoke(msg)
        
        ans_ollama = ans_ollama.content.strip().strip('.').strip()
        
        
        if ans_ollama != ans[0].strip().strip('.').strip() and ctxt.find(ans_ollama) != -1:
            results_nongen['prompt_2'][temp] += 1
            print(results_nongen)

  0%|          | 0/154 [00:00<?, ?it/s]

{'prompt_1': {0.1: 0, 0.2: 0, 0.3: 0}, 'prompt_2': {0.1: 0, 0.2: 0, 0.3: 1}}
{'prompt_1': {0.1: 0, 0.2: 0, 0.3: 1}, 'prompt_2': {0.1: 0, 0.2: 0, 0.3: 1}}
{'prompt_1': {0.1: 0, 0.2: 0, 0.3: 1}, 'prompt_2': {0.1: 0, 0.2: 1, 0.3: 1}}
{'prompt_1': {0.1: 0, 0.2: 0, 0.3: 1}, 'prompt_2': {0.1: 0, 0.2: 1, 0.3: 2}}
{'prompt_1': {0.1: 0, 0.2: 0, 0.3: 1}, 'prompt_2': {0.1: 0, 0.2: 2, 0.3: 2}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 0, 0.2: 2, 0.3: 2}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 2}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 1, 0.2: 2, 0.3: 3}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 2, 0.2: 2, 0.3: 3}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 2, 0.2: 2, 0.3: 4}}
{'prompt_1': {0.1: 0, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 2, 0.2: 3, 0.3: 4}}
{'prompt_1': {0.1: 1, 0.2: 1, 0.3: 1}, 'prompt_2': {0.1: 2, 0.2: 3, 0.3: 4}}
{'prompt_1': {0.1: 1, 0.2: 2, 0.3: 1}, 'prompt_2': {0.1: 2, 0.2: 3, 0.3: 4}}

In [26]:
print(results_nongen)

{'prompt_1': {0.1: 2, 0.2: 2, 0.3: 2}, 'prompt_2': {0.1: 3, 0.2: 5, 0.3: 4}}
